# Teleportation #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.misc import *;
from src.thirdparty.quantum import *;
from src.thirdparty.render import *;

from src.api.ibm import *;
from src.algorithms import *;

np.random.seed(39102901); # for repeatability

In [ ]:
# Display Teleportation Circuit
with CreateBackend(
    kind = BACKEND_SIMULATOR.AER,
) as backend:
    print('Quantumcircuit for Teleportation:');
    circuit = teleportation_protocol(include_entanglement=False);
    display(circuit.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=False));

In [ ]:
# Run test of teleportation protocol
NUM_SAMPLE = 100; # <- do not set this too high!
NUM_SHOTS = 100;
with CreateBackend(
    kind = BACKEND_SIMULATOR.AER,
) as backend:
    print('Quantumcircuit for testing Teleportation protocol:');
    circuit_scheme, params = teleportation_protocol_test();
    display(circuit_scheme.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=True));

    print(dedent('''
    \x1b[1mNOTE:\x1b[0m
    1) Random state \x1b[1m|ψ⟩ = U|0⟩\x1b[0m generated at start.
       To test that \x1b[1m|ψ⟩\x1b[0m was teleported,
       Bob applies the inverse of \x1b[1mU\x1b[0m and measures his state.
       If \x1b[1mcbit 3 = |0⟩\x1b[0m, then teleportation successful.
    2) We expect cbits 1+2 to be uniformly randomly distributed.
    '''));
    
    # instantiate schema with random values:
    circuits = [
        circuit_scheme.bind_parameters({
            param_i: value_i
            for param_i, value_i in zip(params, values)
        })
        for values in random_unitary_parameters(n=NUM_SAMPLE)
    ];

    # transpile + assemble
    circuits = qk_assemble([ qk_transpile(circuit, backend) for circuit in circuits ]);

    # run job and obtain results:
    job = backend.run(circuits, shots=NUM_SHOTS);
    result = job.result();

    # extract statitics:
    # NOTE: qiskit orders the measured bits from buttom to top, so reverse this.
    counts = {};
    counts_12 = {};
    counts_3 = {};
    for count in result.get_counts():
        for key, value in count.items():
            counts[key] = counts.get(key, 0) + value;
    counts = { key[::-1]: value for key, value in counts.items() };
    counts_12 = {
        key: sum([value for key_, value in counts.items() if key_[:2] == key])
        for key in ['00', '01', '10', '11']
    };
    counts_3 = {
        key: sum([value for key_, value in counts.items() if key_[2] == key])
        for key in ['0', '1']
    };

    display(QkVisualisation.plot_histogram(counts_12, title='Measurements of 1st + 2nd QBit (Alice)'));
    display(QkVisualisation.plot_histogram(counts_3, title='Measurements of 3rd QBit (Bob)'));
